In [ ]:
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#A code to run ordinary least squares with associated statistics
#Jeremy Kedziora
#24 March 2016
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

#import libraries
import numpy as np    #import for arrays
from scipy.optimize import minimize    #import for optimization


In [ ]:
#define functions
def maker(N,n_vars,kind = 'linear'):
    """A function to generate Monte Carlo linear regression data"""
    x = []    #an empty list to hold the data
    y = np.zeros(N)    #an array to hold the dependent variable
    b = []    #an empty list to hold the true bs
    i = 1
    while i <= n_vars:    #loop over the variables we want to create
        x_i = np.random.normal(loc = 0.0, scale = 1.0, size = N)    #generate the data
        x.append(x_i)    #add it to the list of data
        b_i = np.random.normal(loc = 0.0, scale = 1.0)    #draw a random effect for this variable
        b.append(b_i)    #add it to the list of effects
        y = y + b_i*x_i    #add the variable effect to the dependent variable
        i += 1    #index up i
    
    x.append(np.ones(N))    #and a column of ones for a constant
    b_i = np.random.normal(loc = 0.0, scale = 1.0)    #draw a random intercept
    b.append(b_i)    #append this intercept to the effects
    if kind == 'linear':
        y = b_i + y + np.random.normal(loc = 0.0, scale = 1.0, size = N)    #add the normally distributed error term and the intercept
    if kind == 'logit':
        y = (np.random.uniform(0,1,len(y)) < np.exp(b_i + y)/(1 + np.exp(b_i + y)))*1
    return [np.array(x).T,np.array(y),np.array(b)]

In [ ]:
def OLS_mle(b,X,y):
    """A function to compute OLS coefficients using MLE"""
    s2 = 1.0#math.exp(b[len(b) - 1])    #exponentiate the variance to ensure that it is positive
    xb = X.dot(b)    #compute the means
    return -1*sum(-0.5*np.log(s2) - (y - xb)**2/(2*s2))    #return the log likelihood

In [ ]:
def logit_mle(b,X,y):
    """A function to compute logit coefficients using MLE"""
    xb = X.dot(b)    #compute the means
    return -1*sum(y*xb - np.log(1+np.exp(xb)))    #return the log likelihood

In [ ]:
Data = maker(100000,3,kind = 'logit')    #make logit data
X = Data[0]    #pull out explanatory variables
y = Data[1]    #pull out dependent variable
b = Data[2]    #pull out true coefficients

b = np.random.uniform(0,1,4)*0.01    #set starting values
Coefficients = minimize(logit_mle, x0 = b, args = (X,y), method = 'BFGS').x    #maximize the log-likelihood
print(Coefficients)    #print out the coefficients
print(Data[2])    #print out the true betas

In [ ]:
Data = maker(100,3,kind = 'linear')    #make logit data
X = Data[0]    #pull out explanatory variables
y = Data[1]    #pull out dependent variable
b = Data[2]    #pull out true coefficients

b = np.random.uniform(0,1,4)*0.01    #set starting values
Coefficients = minimize(OLS_mle, x0 = b, args = (X,y), method = 'BFGS').x    #maximize the log-likelihood
print(Coefficients)
print(Data[2])